## Part 1 - Planning problems

**load_actions** in AirCargoProblem.get_actions method (my_air_cargo_problems.py)

In [ ]:
def load_actions():
    """
    Create all concrete Load actions and return a list

    :return: list of Action objects
    """
    loads = []
    # TODO create all load ground actions from the domain Load action
    # Action(Load(c, p, a),
    #	PRECOND: At(c, a) ∧ At(p, a) ∧ Cargo(c) ∧ Plane(p) ∧ Airport(a) 
    #     전제조건 : C는 A에 있다. P는 A에 있다. C, P, A가 존재한다.
    #	EFFECT: ¬ At(c, a) ∧ In(c, p)) 
    #     결과 : C(수화물)은 A(공항)에 없고 P(비행기)에 있게 된다.

    for c in self.cargos: #수화물
        for p in self.planes: #비행기
            for a in self.airports: #공항. 각 루프에서 필요한 요소들을 가져온다.
                precond_pos = [expr("At({}, {})".format(c, a)),
                               expr("At({}, {})".format(p, a))] #positive 전제조건 리스트
                precond_neg = [] #negative 전제조건 리스트
                effect_add = [expr("In({}, {})".format(c, p))] #positive 결과 리스트
                effect_rem = [expr("At({}, {})".format(c, a))] #negative 결과 리스트
                load = Action(expr("Load({}, {}, {})".format(c, p, a)),
                              [precond_pos, precond_neg],
                              [effect_add, effect_rem]) #LOAD(c, p, a)
                loads.append(load)
                
    return loads

**unload_actions** in AirCargoProblem.get_actions method (my_air_cargo_problems.py)

In [ ]:
def unload_actions():
    """
    Create all concrete Unload actions and return a list

    :return: list of Action objects
    """
    unloads = []
    # TODO create all Unload ground actions from the domain Unload action
    # Action(Unload(c, p, a),
    #	PRECOND: In(c, p) ∧ At(p, a) ∧ Cargo(c) ∧ Plane(p) ∧ Airport(a)
    #     전제조건 : C는 P에 있다. P는 A에 있다. C, P, A가 존재한다.
    #	EFFECT: At(c, a) ∧ ¬ In(c, p))
    #     결과 : C(수화물)은 A(공항)에 있고 P(비행기)에 없게 된다.

    for c in self.cargos: #수화물
        for p in self.planes: #비행기
            for a in self.airports: #공항. 각 루프에서 필요한 것들을 가져온다.
                precond_pos = [expr("In({}, {})".format(c, p)),
                               expr("At({}, {})".format(p, a))] #positive 전제조건 리스트
                precond_neg = [] #negative 전제조건 리스트
                effect_add = [expr("At({}, {})".format(c, a))] #positive 결과 리스트
                effect_rem = [expr("In({}, {})".format(c, p))] #negative 결과 리스트
                unload = Action(expr("Unload({}, {}, {})".format(c, p, a)),
                    [precond_pos, precond_neg],
                    [effect_add, effect_rem]) #Unload(c, p, a)
                unloads.append(unload)
                
    return unloads

**AirCargoProblem.actions** (my_air_cargo_problems.py)

In [ ]:
def actions(self, state: str) -> list: #Python 3 부터 파라미터의 자료형과 반환형을 지정해 줄 수 있다(Swift 처럼). 
    #하지만 일치하지 않아도 에러를 발생시키지는 않는다. 일종의 주석이라 보면 될 듯.
    """ 
    Return the actions that can be executed in the given state.

    :param state: str
        state represented as T/F string of mapped fluents (state variables)
        e.g. 'FTTTFF'
    :return: list of Action objects
    """
    # TODO implement
    possible_actions = [] #반환 리스트
    kb = PropKB() #명제논리를 위한 로직 중 하나.
    kb.tell(decode_state(state, self.state_map).pos_sentence()) #sentence's clauses 추가
    for action in self.actions_list:
        is_possible = True
        for clause in action.precond_pos: #positive 전제조건
            if clause not in kb.clauses: #postive가 없으면 False
                is_possible = False
        for clause in action.precond_neg: #negative 전제조건
            if clause in kb.clauses: #negative가 있으면 Flase
                is_possible = False
        if is_possible: #가능한 조건이라면 추가
            possible_actions.append(action)

    return possible_actions

**AirCargoProblem.result** (my_air_cargo_problems.py)

In [ ]:
def result(self, state: str, action: Action): #Python 3 부터 파라미터의 자료형과 반환형을 지정해 줄 수 있다(Swift 처럼). 
    #하지만 일치하지 않아도 에러를 발생시키지는 않는다. 일종의 주석이라 보면 될 듯.
    """ 
    Return the state that results from executing the given
    action in the given state. The action must be one of
    self.actions(state).

    :param state: state entering node
    :param action: Action applied
    :return: resulting state after action
    """
    # TODO implement
    new_state = FluentState([], [])
    old_state = decode_state(state, self.state_map)
    for fluent in old_state.pos: #positive oldstate가 
        if fluent not in action.effect_rem: #negative 결과에 없다면
            new_state.pos.append(fluent) #positive newstate에 추가
    for fluent in action.effect_add: #positive 결과가
        if fluent not in new_state.pos: #positive oldstate에 없다면
            new_state.pos.append(fluent) #positive newstate에 추가
    for fluent in old_state.neg: #negative oldstate가
        if fluent not in action.effect_add: #positive 결과에 없다면
            new_state.neg.append(fluent) #negative newstate에 추가
    for fluent in action.effect_rem: #negative 결과가
        if fluent not in new_state.neg: #negative newstate에 없다면
            new_state.neg.append(fluent) #negative newstate에 추가

    return encode_state(new_state, self.state_map)

**air_cargo_p2** (my_air_cargo_problems.py)

In [ ]:
def air_cargo_p2() -> AirCargoProblem:
    # TODO implement Problem 2 definition
    # Init(At(C1, SFO) ∧ At(C2, JFK) ∧ At(C3, ATL)
    # 	∧ At(P1, SFO) ∧ At(P2, JFK) ∧ At(P3, ATL)
    # 	∧ Cargo(C1) ∧ Cargo(C2) ∧ Cargo(C3)
    # 	∧ Plane(P1) ∧ Plane(P2) ∧ Plane(P3)
    # 	∧ Airport(JFK) ∧ Airport(SFO) ∧ Airport(ATL))
    # Goal(At(C1, JFK) ∧ At(C2, SFO) ∧ At(C3, SFO))

    cargos = ["C1", "C2", "C3"]
    planes = ["P1", "P2", "P3"]
    airports = ["JFK", "SFO", "ATL"]
    pos = [expr("At(C1, SFO)"),
           expr("At(C2, JFK)"),
           expr("At(C3, ATL)"),
           expr("At(P1, SFO)"),
           expr("At(P2, JFK)"),
           expr("At(P3, ATL)")]
    neg = [expr("At(C1, JFK)"),
           expr("At(C1, ATL)"),
           expr("In(C1, P1)"),
           expr("In(C1, P2)"),
           expr("In(C1, P3)"),
           expr("At(C2, SFO)"),
           expr("At(C2, ATL)"),
           expr("In(C2, P1)"),
           expr("In(C2, P2)"),
           expr("In(C2, P3)"),
           expr("At(C3, JFK)"),
           expr("At(C3, SFO)"),
           expr("In(C3, P1)"),
           expr("In(C3, P2)"),
           expr("In(C3, P3)"),
           expr("At(P1, JFK)"),
           expr("At(P1, ATL)"),
           expr("At(P2, SFO)"),
           expr("At(P2, ATL)"),
           expr("At(P3, JFK)"),
           expr("At(P3, SFO)"),
          ]
    init = FluentState(pos, neg)
    goal = [expr("At(C1, JFK)"),
            expr("At(C2, SFO)"),
            expr("At(C3, SFO)")]

    return AirCargoProblem(cargos, planes, airports, init, goal)

**air_cargo_p3** (my_air_cargo_problems.py)

In [ ]:
def air_cargo_p3() -> AirCargoProblem:
    # TODO implement Problem 3 definition
    # Init(At(C1, SFO) ∧ At(C2, JFK) ∧ At(C3, ATL) ∧ At(C4, ORD)
    # 	∧ At(P1, SFO) ∧ At(P2, JFK)
    # 	∧ Cargo(C1) ∧ Cargo(C2) ∧ Cargo(C3) ∧ Cargo(C4)
    # 	∧ Plane(P1) ∧ Plane(P2)
    # 	∧ Airport(JFK) ∧ Airport(SFO) ∧ Airport(ATL) ∧ Airport(ORD))
    # Goal(At(C1, JFK) ∧ At(C3, JFK) ∧ At(C2, SFO) ∧ At(C4, SFO))

    cargos = ["C1", "C2", "C3", "C4"]
    planes = ["P1", "P2"]
    airports = ["JFK", "SFO", "ATL", "ORD"]
    pos = [expr("At(C1, SFO)"),
           expr("At(C2, JFK)"),
           expr("At(C3, ATL)"),
           expr("At(C4, ORD)"),
           expr("At(P1, SFO)"),
           expr("At(P2, JFK)")]
    neg = [expr("At(C1, JFK)"),
           expr("At(C1, ATL)"),
           expr("At(C1, ORD)"),
           expr("In(C1, P1)"),
           expr("In(C1, P2)"),
           expr("At(C2, SFO)"),
           expr("At(C2, ATL)"),
           expr("At(C2, ORD)"),
           expr("In(C2, P1)"),
           expr("In(C2, P2)"),
           expr("At(C3, JFK)"),
           expr("At(C3, SFO)"),
           expr("At(C3, ORD)"),
           expr("In(C3, P1)"),
           expr("In(C3, P2)"),
           expr("At(C4, JFK)"),
           expr("At(C4, SFO)"),
           expr("At(C4, ATL)"),
           expr("In(C4, P1)"),
           expr("In(C4, P2)"),
           expr("At(P1, JFK)"),
           expr("At(P1, ATL)"),
           expr("At(P1, ORD)"),
           expr("At(P2, SFO)"),
           expr("At(P2, ATL)"),
           expr("At(P2, ORD)"),
          ]
    init = FluentState(pos, neg)
    goal = [expr("At(C1, JFK)"),
            expr("At(C3, JFK)"),
            expr("At(C2, SFO)"),
            expr("At(C4, SFO)")]

    return AirCargoProblem(cargos, planes, airports, init, goal)

## Part 2 - Domain-independent heuristics

** h_ignore_preconditions ** (my_air_cargo_problems.py)

In [ ]:
@lru_cache(maxsize=8192) #@는 데코레이터. 기존에 정의된 함수를 확장한다. 데코레이터 역할을 하는 함수를 직접 추가해 줄 수도 있다. 
#http://jonnung.github.io/python/2015/08/17/python-decorator/
def h_ignore_preconditions(self, node: Node):
    """
    This heuristic estimates the minimum number of actions that must be
    carried out from the current state in order to satisfy all of the goal
    conditions by ignoring the preconditions required for an action to be
    executed.
    """
    # TODO implement (see Russell-Norvig Ed-3 10.2.3  or Russell-Norvig Ed-2 11.2)
    kb = PropKB()
    kb.tell(decode_state(node.state, self.state_map).pos_sentence()) #sentence's clauses 추가
    count = 0
    for clause in self.goal:
        if clause not in kb.clauses: #속해있지 않으면 증가. 현재 상태에서 수행해야할 작업 수를 계산한다. 
            count += 1
            
    return count

----

In [ ]:
class PgNode(): #노드
    """
    Base class for planning graph nodes.

    includes instance sets common to both types of nodes used in a planning graph
    parents: the set of nodes in the previous level
    children: the set of nodes in the subsequent level
    mutex: the set of sibling nodes that are mutually exclusive with this node
    """

    def __init__(self):
        self.parents = set()
        self.children = set()
        self.mutex = set()

    def is_mutex(self, other) -> bool: #뮤텍스 판별
        """
        Boolean test for mutual exclusion

        :param other: PgNode
            the other node to compare with
        :return: bool
            True if this node and the other are marked mutually exclusive (mutex)
        """
        if other in self.mutex:
            return True
        return False

    def show(self):
        """
        helper print for debugging shows counts of parents, children, siblings

        :return:
            print only
        """
        print("{} parents".format(len(self.parents)))
        print("{} children".format(len(self.children)))
        print("{} mutex".format(len(self.mutex)))

In [ ]:
class PgNode_s(PgNode): #상태 노드
    """
    A planning graph node representing a state (literal fluent) from a
    planning problem.

    Args:
    ----------
    symbol : str
        A string representing a literal expression from a planning problem
        domain.

    is_pos : bool
        Boolean flag indicating whether the literal expression is positive or
        negative.
    """

    def __init__(self, symbol: str, is_pos: bool):
        """
        S-level Planning Graph node constructor

        :param symbol: expr
        :param is_pos: bool
        Instance variables calculated:
            literal: expr
                    fluent in its literal form including negative operator if applicable
        Instance variables inherited from PgNode:
            parents: set of nodes connected to this node in previous A level; initially empty
            children: set of nodes connected to this node in next A level; initially empty
            mutex: set of sibling S-nodes that this node has mutual exclusion with; initially empty
        """
        PgNode.__init__(self)
        self.symbol = symbol
        self.is_pos = is_pos
        self.__hash = None

    def show(self):
        """
        helper print for debugging shows literal plus counts of parents,
        children, siblings

        :return:
            print only
        """
        if self.is_pos:
            print("\n*** {}".format(self.symbol))
        else:
            print("\n*** ~{}".format(self.symbol))
        PgNode.show(self)

    def __eq__(self, other):
        """equality test for nodes - compares only the literal for equality

        :param other: PgNode_s
        :return: bool
        """
        return (isinstance(other, self.__class__) and
                self.is_pos == other.is_pos and
                self.symbol == other.symbol)

    def __hash__(self):
        self.__hash = self.__hash or hash(self.symbol) ^ hash(self.is_pos)
        return self.__hash

In [ ]:
class PgNode_a(PgNode): #Action 노드
    """A-type (action) Planning Graph node - inherited from PgNode """


    def __init__(self, action: Action): 
        """A-level Planning Graph node constructor

        :param action: Action
            a ground action, i.e. this action cannot contain any variables
        Instance variables calculated:
            An A-level will always have an S-level as its parent and an S-level as its child.
            The preconditions and effects will become the parents and children of the A-level node
            However, when this node is created, it is not yet connected to the graph
            prenodes: set of *possible* parent S-nodes
            effnodes: set of *possible* child S-nodes
            is_persistent: bool   True if this is a persistence action, i.e. a no-op action
        Instance variables inherited from PgNode:
            parents: set of nodes connected to this node in previous S level; initially empty
            children: set of nodes connected to this node in next S level; initially empty
            mutex: set of sibling A-nodes that this node has mutual exclusion with; initially empty
        """
        #A 레벨은 항상 S 레벨을 상위 레벨로, S 레벨을 하위 레벨로 가진다. (두 S 레벨 사이에 A레벨이 있다.)
        #전제조건은 A노드의 부모, 효과는 A노드의 자식이 된다. 노드 생성시에는 관계가 형성되어 있지 않으므로 뒤에 설정해 주어야 한다.        
        PgNode.__init__(self) #부모 클래스의 생성자 호출
        self.action = action #액션 설정
        self.prenodes = self.precond_s_nodes() #가능한 부모 s노드 집합
        self.effnodes = self.effect_s_nodes() #가능한 자식 s노드 집합
        self.is_persistent = self.prenodes == self.effnodes #지속 가능 여부. 자식과 부모가 같다면 True
        self.__hash = None

    def show(self):
        """helper print for debugging shows action plus counts of parents, children, siblings

        :return:
            print only
        """
        print("\n*** {!s}".format(self.action))
        PgNode.show(self)

    def precond_s_nodes(self):
        """precondition literals as S-nodes (represents possible parents for this node).
        It is computationally expensive to call this function; it is only called by the
        class constructor to populate the `prenodes` attribute.

        :return: set of PgNode_s
        """
        nodes = set()
        for p in self.action.precond_pos:
            nodes.add(PgNode_s(p, True))
        for p in self.action.precond_neg:
            nodes.add(PgNode_s(p, False))
        return nodes

    def effect_s_nodes(self):
        """effect literals as S-nodes (represents possible children for this node).
        It is computationally expensive to call this function; it is only called by the
        class constructor to populate the `effnodes` attribute.

        :return: set of PgNode_s
        """
        nodes = set()
        for e in self.action.effect_add:
            nodes.add(PgNode_s(e, True))
        for e in self.action.effect_rem:
            nodes.add(PgNode_s(e, False))
        return nodes

    def __eq__(self, other):
        """equality test for nodes - compares only the action name for equality

        :param other: PgNode_a
        :return: bool
        """
        return (isinstance(other, self.__class__) and
                self.is_persistent == other.is_persistent and
                self.action.name == other.action.name and
                self.action.args == other.action.args)

    def __hash__(self):
        self.__hash = self.__hash or hash(self.action.name) ^ hash(self.action.args)
        return self.__hash

In [ ]:
def create_graph(self):
    """ 
    build a Planning Graph as described in Russell-Norvig 3rd Ed 10.3 or 2nd Ed 11.4

    The S0 initial level has been implemented for you.  It has no parents and includes all of
    the literal fluents that are part of the initial state passed to the constructor.  At the start
    of a problem planning search, this will be the same as the initial state of the problem.  However,
    the planning graph can be built from any state in the Planning Problem

    This function should only be called by the class constructor.

    :return:
        builds the graph by filling s_levels[] and a_levels[] lists with node sets for each level
    """
    # the graph should only be built during class construction
    if (len(self.s_levels) != 0) or (len(self.a_levels) != 0):
        raise Exception(
            'Planning Graph already created; construct a new planning graph for each new state in the planning sequence')

    # initialize S0 to literals in initial state provided.
    #상태 0을 초기화.
    leveled = False
    level = 0 #레벨
    self.s_levels.append(set())  # S0 set of s_nodes - empty to start
    # for each fluent in the initial state, add the correct literal PgNode_s
    for literal in self.fs.pos: #positive 중에서 loop
        self.s_levels[level].add(PgNode_s(literal, True)) #PgNode_s : state type 그래프 노드. init(symbol, is_pos)
        #s_levels는 각 레벨(인덱스)에 PgNode_s로 추가. 위에서 set으로 설정했기에 중복이 없다.
    for literal in self.fs.neg: #negative 중에서 loop
        self.s_levels[level].add(PgNode_s(literal, False)) #PgNode_s : state type 그래프 노드. init(symbol, is_pos)
        #s_levels는 각 레벨(인덱스)에 PgNode_s로 추가. 위에서 set으로 설정했기에 중복이 없다.
    # no mutexes at the first level
    #첫 레벨에서는 뮤텍스가 없다. 뮤텍스 : 함께 표시될 수 없는 리터럴

    # continue to build the graph alternating A, S levels until last two S levels contain the same literals,
    # i.e. until it is "leveled"
    while not leveled: #planning Graph대로 레벨을 증가시키면서 액션과 상태 뮤텍스 추가.
        self.add_action_level(level) #action 추가
        self.update_a_mutex(self.a_levels[level]) #mutex 추가

        level += 1 #레벨 증가
        self.add_literal_level(level)
        self.update_s_mutex(self.s_levels[level])

        if self.s_levels[level] == self.s_levels[level - 1]: #이전과 상태가 같아지면 종료.
            #상태가 같으면, 거기서 나올 수 있는 액션과 상태 뮤텍스 또한 반복되므로 종료할 수 있다.
            leveled = True

----
** add_action_level ** (my_planning_graph.py)

In [ ]:
def add_action_level(self, level): #A 레벨 추가
    """ 
    add an A (action) level to the Planning Graph

    :param level: int
        the level number alternates S0, A0, S1, A1, S2, .... etc the level number is also used as the
        index for the node set lists self.a_levels[] and self.s_levels[]
    :return:
        adds A nodes to the current level in self.a_levels[level]
    """
    # TODO add action A level to the planning graph as described in the Russell-Norvig text
    # 1. determine what actions to add and create those PgNode_a objects
    # 2. connect the nodes to the previous S literal level
    # for example, the A0 level will iterate through all possible actions for the problem and add a PgNode_a to a_levels[0]
    #   set iff all prerequisite literals for the action hold in S0.  This can be accomplished by testing
    #   to see if a proposed PgNode_a has prenodes that are a subset of the previous S level.  Once an
    #   action node is added, it MUST be connected to the S node instances in the appropriate s_level set.
    
    self.a_levels.append(set()) #초기화
    
    for action in self.all_actions:
        node_a = PgNode_a(action) #PgNode_a: action type 그래프 노드
        pre_s_level = self.s_levels[level] #상위 S 레벨 불러온다.
        
        if node_a.prenodes.issubset(pre_s_level): #가능한 부모노드(S) 셋이 이전 레벨 S의 서브셋이라면(prenodes는 '가능한' 부모 노드의 집합.)
            for node_s in pre_s_level: #노드의 관계를 설정한다.
                node_s.children.add(node_a) #S - A
                node_a.parents.add(node_s) #A - S
                #S - A - S의 형태로 연결되어야 하므로.
        
            self.a_levels[level].add(node_a) #Action 레벨 리스트에 추가(요소는 인덱스(레벨)의 셋으로 설정되어 있다.)        

** add_literal_level ** (my_planning_graph.py)

In [ ]:
def add_literal_level(self, level): #S 레벨 추가
    """ 
    add an S (literal) level to the Planning Graph

    :param level: int
        the level number alternates S0, A0, S1, A1, S2, .... etc the level number is also used as the
        index for the node set lists self.a_levels[] and self.s_levels[]
    :return:
        adds S nodes to the current level in self.s_levels[level]
    """
    # TODO add literal S level to the planning graph as described in the Russell-Norvig text
    # 1. determine what literals to add
    # 2. connect the nodes
    # for example, every A node in the previous level has a list of S nodes in effnodes that represent the effect
    #   produced by the action.  These literals will all be part of the new S level.  Since we are working with sets, they
    #   may be "added" to the set without fear of duplication.  However, it is important to then correctly create and connect
    #   all of the new S nodes as children of all the A nodes that could produce them, and likewise add the A nodes to the
    #   parent sets of the S nodes
    
    pre_a_level = self.a_levels[level - 1] # S - A - S로 연결되어야 하므로, 이전 A 레벨을 가져와야 한다.
    self.s_levels.append(set()) #초기화
    
    for node_a in pre_a_level: #이전 A 레벨의 노드들 가져오기
        for node_s in node_a.effnodes: #해당 A노드의 자식 노드가 될 수 있는 S노드들. Effect가 된다.
            node_s.parents.add(node_a) #S - A
            node_a.children.add(node_s) #A - S
            #S - A - S의 형태로 연결되어야 하므로.
            self.s_levels[level].add(node_s) #State 레벨 리스트에 추가(요소는 인덱스(레벨)의 셋으로 설정되어 있다.)     

** inconsistent_effects_mutex ** (my_planning_graph.py)

In [ ]:
def inconsistent_effects_mutex(self, node_a1: PgNode_a, node_a2: PgNode_a) -> bool:
    """
    Test a pair of actions for inconsistent effects, returning True if
    one action negates an effect of the other, and False otherwise.

    HINT: The Action instance associated with an action node is accessible
    through the PgNode_a.action attribute. See the Action class
    documentation for details on accessing the effects and preconditions of
    an action.

    :param node_a1: PgNode_a
    :param node_a2: PgNode_a
    :return: bool
    """
    # TODO test for Inconsistent Effects between nodes
    
    #Inconsistent effects: one action negates an effect of the other. 
    #For example Eat(Cake) and the persistence of Have(Cake) have inconsistent effects 
    #because they disagree on the effect Have(Cake).

    for effect in node_a1.action.effect_add: #Positive 결과
        if effect in node_a2.action.effect_rem: #Negative 결과
            return True

    for effect in node_a2.action.effect_add: #Positive 결과
        if effect in node_a1.action.effect_rem: #Negative 결과
            return True

    return False

** interference_mutex ** (my_planning_graph.py)

In [ ]:
def interference_mutex(self, node_a1: PgNode_a, node_a2: PgNode_a) -> bool:
    """
    Test a pair of actions for mutual exclusion, returning True if the 
    effect of one action is the negation of a precondition of the other.

    HINT: The Action instance associated with an action node is accessible
    through the PgNode_a.action attribute. See the Action class
    documentation for details on accessing the effects and preconditions of
    an action.

    :param node_a1: PgNode_a
    :param node_a2: PgNode_a
    :return: bool
    """
    # TODO test for Interference between nodes
    #Interference: one of the effects of one action is the negation of a precondition of the other. 
    #For example Eat(Cake) interferes with the persistence of Have(Cake) by negat- ing its precondition.
    
    for effect in node_a1.action.effect_add: #Positive 결과
        if effect in node_a2.action.precond_neg: #Negative 전제 조건
            return True
            
    for effect in node_a2.action.effect_add: #Positive 결과
        if effect in node_a1.action.precond_neg: #Negative 전제 조건
            return True
        
    for effect in node_a1.action.effect_rem: #Negative 결과
        if effect in node_a2.action.precond_pos: #Positive 전제 조건
            return True

    for effect in node_a2.action.effect_rem:#Negative 결과
        if effect in node_a1.action.precond_pos: #Positive 전제 조건
            return True
                
    return False

** competing_needs_mutex ** (my_planning_graph.py)

In [ ]:
def competing_needs_mutex(self, node_a1: PgNode_a, node_a2: PgNode_a) -> bool:
    """
    Test a pair of actions for mutual exclusion, returning True if one of
    the precondition of one action is mutex with a precondition of the
    other action.

    :param node_a1: PgNode_a
    :param node_a2: PgNode_a
    :return: bool
    """

    # TODO test for Competing Needs between nodes
    
    #Competing needs: one of the preconditions of one action is mutually exclusive with a precondition of the other. 
    #For example, Bake(Cake) and Eat(Cake) are mutex because they compete on the value of the Have(Cake) precondition.
    
    for a1_precond in node_a1.parents:
        for a2_precond in node_a2.parents:
            if a1_precond.is_mutex(a2_precond): #뮤텍스 판별
                return True
            
    return False

** negation_mutex ** (my_planning_graph.py)

In [ ]:
def negation_mutex(self, node_s1: PgNode_s, node_s2: PgNode_s) -> bool:
    """
    Test a pair of state literals for mutual exclusion, returning True if
    one node is the negation of the other, and False otherwise.

    HINT: Look at the PgNode_s.__eq__ defines the notion of equivalence for
    literal expression nodes, and the class tracks whether the literal is
    positive or negative.

    :param node_s1: PgNode_s
    :param node_s2: PgNode_s
    :return: bool
    """
    # TODO test for negation between nodes
    return node_s1.symbol == node_s2.symbol and node_s1.is_pos != node_s2.is_pos   

** inconsistent_support_mutex ** (my_planning_graph.py)

In [ ]:
def inconsistent_support_mutex(self, node_s1: PgNode_s, node_s2: PgNode_s):
    """
    Test a pair of state literals for mutual exclusion, returning True if
    there are no actions that could achieve the two literals at the same
    time, and False otherwise.  In other words, the two literal nodes are
    mutex if all of the actions that could achieve the first literal node
    are pairwise mutually exclusive with all of the actions that could
    achieve the second literal node.

    HINT: The PgNode.is_mutex method can be used to test whether two nodes
    are mutually exclusive.

    :param node_s1: PgNode_s
    :param node_s2: PgNode_s
    :return: bool
    """
    # TODO test for Inconsistent Support between nodes
    for s1_action in node_s1.parents:
        for s2_action in node_s2.parents:
            if not s1_action.is_mutex(s2_action):
                return False

    return True

** h_levelsum ** (my_planning_graph.py)

In [ ]:
def h_levelsum(self) -> int:
    """
    The sum of the level costs of the individual goals (admissible if goals independent)

    :return: int
    """
    level_sum = 0
    # TODO implement
    # for each goal in the problem, determine the level cost, then add them together
    goals = self.problem.goal
    s_levels = self.s_levels
    for goal in goals:
        node = PgNode_s(goal, True)
        s_levels_list = enumerate(s_levels)
        for level, s_nodes in s_levels_list:
            if node in s_nodes:
                level_sum += level
                break
    return level_sum